In [ ]:
%load_ext autoreload
%autoreload 2

In [12]:
import math

import torch
from allennlp.data.token_indexers import PretrainedBertIndexer
from allennlp.data.tokenizers import WordTokenizer
from allennlp.data.tokenizers.word_splitter import BertBasicWordSplitter
from pytorch_pretrained_bert import BertTokenizer, BertForMaskedLM
from tqdm import tqdm_notebook
from tuple_filtering.tuple_dataset_reader import TupleDatasetReader, ConcatenatedTupleDatasetReader

In [2]:
from allennlp.modules.token_embedders.bert_token_embedder import PretrainedBertEmbedder
from allennlp.data.instance import Instance

In [3]:
def tuple_instance_to_array(instance):
    [[str(token) for token in tf.tokens] for _, tf in instance.items()]

In [13]:
bert_type = 'bert-base-uncased'

token_indexers = {"tokens": PretrainedBertIndexer(pretrained_model=bert_type)}
tokenizer = WordTokenizer(word_splitter=BertBasicWordSplitter())

reader = TupleDatasetReader(lazy=True, tokenizer=tokenizer, token_indexers=token_indexers)
tuple_dataset = reader.read('data/openbook_tuples.tsv')

instance = next(iter(tuple_dataset))

In [11]:
instance.fields["object_tokens"].tokens

[flow, of, electricity]

In [ ]:
bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def get_score(sentence: str):
    tokenize_input = bert_tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([bert_tokenizer.convert_tokens_to_ids(tokenize_input)])
    predictions = bert_model(tensor_input)
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(predictions.squeeze(), tensor_input.squeeze()).data
    return math.exp(loss)

# def sorted_tuple_scores(tuple_data_path: str):
#     scores = []
#     with open(tuple_data_path, 'r') as f:
#         for line in tqdm_notebook(f.readlines()):
#             sentence = " ".join(line.strip().split('\t'))
#             scores.append((sentence, get_score(sentence)))
#     scores.sort(key=lambda x: x[1])
#     return scores

In [ ]:
# aristo_scores = sorted_tuple_scores('data/aristo.txt')
# display(aristo_scores)

In [ ]:
# openbook_scores = sorted_tuple_scores('data/openbook_tuples.tsv')
# display(openbook_scores)